<a href="https://colab.research.google.com/github/abxda/COLMEX-ML/blob/main/Semana_06_COLMEX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas geopandas seaborn tqdm requests loguru geohexgrid --quiet

In [ ]:
"""
Script para integrar datos censales y geográficos de 32 estados de México en DuckDB,
procesando archivos CSV y shapefiles para generar:
  - La tabla 'censo_geo_int' con datos censales, con campos numéricos convertidos a INTEGER.
  - La tabla 'hex_grid_1km_4326' que contiene una grilla hexagonal calculada con geohexgrid.
"""

In [ ]:

import os
import time
import shutil
from zipfile import ZipFile
import requests
from tqdm import tqdm
import duckdb
import geopandas as gpd
import pandas as pd
import chardet
import geohexgrid as ghg
from shapely import wkb


In [ ]:


# --------------------------------------------------
# 1. Funciones Base
# --------------------------------------------------

def download(url, directory):
    """Descarga un archivo desde la URL indicada en 'directory' (salta si ya existe)."""
    filename = url.split('/')[-1]
    filepath = os.path.join(directory, filename)
    if os.path.exists(filepath):
        print(f"El archivo {filename} ya existe, no se descarga de nuevo.")
        return filepath
    print(f"Descargando {filename} ...")
    time.sleep(1)
    r = requests.get(url, stream=True)
    total_size = int(r.headers.get('content-length', 0))
    os.makedirs(directory, exist_ok=True)
    with open(filepath, 'wb') as f, tqdm(
        total=total_size/1024 if total_size else None, unit='KB', desc=filename
    ) as pbar:
        for data in r.iter_content(1024):
            f.write(data)
            pbar.update(len(data)/1024)
    print(f"Descarga completada: {filename}\n")
    return filepath

def extract_shapefile(geo_zip_list, zip_dir, shp_dir, shape_type):
    """
    Extrae los componentes esenciales del shapefile de cada ZIP en geo_zip_list.
    Se asume que dentro de cada ZIP los archivos siguen la nomenclatura:
      conjunto_de_datos/{CODIGO}{shape_type}.{ext}
    """
    os.makedirs(shp_dir, exist_ok=True)
    for estado_zip in geo_zip_list:
        cod = estado_zip.split('_')[0]
        zip_path = os.path.join(zip_dir, estado_zip)
        shp_files = [
            f'conjunto_de_datos/{cod}{shape_type}.shp',
            f'conjunto_de_datos/{cod}{shape_type}.cpg',
            f'conjunto_de_datos/{cod}{shape_type}.dbf',
            f'conjunto_de_datos/{cod}{shape_type}.prj',
            f'conjunto_de_datos/{cod}{shape_type}.shx'
        ]
        # Verifica si ya están extraídos
        if all(os.path.exists(os.path.join(shp_dir, f)) for f in shp_files):
            print(f"Archivos para {cod} ya extraídos en {shp_dir}.")
            continue
        with ZipFile(zip_path, 'r') as zip_ref:
            for f in shp_files:
                try:
                    zip_ref.extract(f, shp_dir)
                except KeyError:
                    if f.endswith('.cpg'):
                        with open(os.path.join(shp_dir, f), 'w') as out_file:
                            out_file.write("ISO 88591")
                    else:
                        print(f"¡ERROR! Archivo esencial '{f}' no encontrado en {estado_zip}.")
        print(f"Shapefile extraído para {cod} en {shp_dir}.\n")

def convert_to_geoparquet(shp_dir, cod, shape_type="m"):
    """
    Convierte el shapefile de un estado a GeoParquet.
    Se asume que el shapefile se encuentra en:
      {shp_dir}/conjunto_de_datos/{cod}{shape_type}.shp
    """
    shp_path = os.path.join(shp_dir, "conjunto_de_datos", f"{cod}{shape_type}.shp")
    parquet_path = os.path.join(shp_dir, "conjunto_de_datos", f"{cod}{shape_type}.geoparquet")
    if os.path.exists(parquet_path):
        print(f"GeoParquet para {cod} ya existe.")
    else:
        print(f"Convirtiendo {shp_path} a GeoParquet...")
        try:
            gdf = gpd.read_file(shp_path, encoding='ISO-8859-1')
            if gdf.crs is None:
                gdf.set_crs("EPSG:6372", inplace=True)
            gdf = gdf.to_crs("EPSG:4326")
            gdf.to_parquet(parquet_path, index=False)
            print(f"  GeoParquet guardado en: {parquet_path}")
        except Exception as e:
            print(f"Error al convertir {shp_path} a GeoParquet: {e}")
    return parquet_path

In [ ]:

# --------------------------------------------------
# 2. Configuración de Directorios y Listas
# --------------------------------------------------

# Listas de archivos (shapefiles) y números de estados
#estados_geo = [
#    "01_aguascalientes.zip", "02_bajacalifornia.zip", "03_bajacaliforniasur.zip",
#    "04_campeche.zip", "05_coahuiladezaragoza.zip", "06_colima.zip", "07_chiapas.zip",
#    "08_chihuahua.zip", "09_ciudaddemexico.zip", "10_durango.zip", "11_guanajuato.zip",
#    "12_guerrero.zip", "13_hidalgo.zip", "14_jalisco.zip", "15_mexico.zip",
#    "16_michoacandeocampo.zip", "17_morelos.zip", "18_nayarit.zip", "19_nuevoleon.zip",
#    "20_oaxaca.zip", "21_puebla.zip", "22_queretaro.zip", "23_quintanaroo.zip",
#    "24_sanluispotosi.zip", "25_sinaloa.zip", "26_sonora.zip", "27_tabasco.zip",
#    "28_tamaulipas.zip", "29_tlaxcala.zip", "30_veracruzignaciodelallave.zip",
#    "31_yucatan.zip", "32_zacatecas.zip"
#]
#estados_num = list(range(1, 33))


estados_geo = ["09_ciudaddemexico.zip","13_hidalgo.zip", "15_mexico.zip"]
estados_num = [9,13,15]


# Directorios de trabajo
base_dir = "./inegi"
download_dir_csv = os.path.join(base_dir, "ccpvagebmza")
csv_dir = os.path.join(download_dir_csv, "csv")
parquet_csv_dir = os.path.join(download_dir_csv, "parquet")
mgc_dir = os.path.join(base_dir, "mgccpv")

# Directorios para GeoParquet unificados
geo_parquet_unificado = os.path.join(mgc_dir, "geoparquet_unificados")

# Limpieza y creación de directorios
for d in [download_dir_csv, mgc_dir]:
    if os.path.exists(d):
        shutil.rmtree(d)
os.makedirs(csv_dir, exist_ok=True)
os.makedirs(parquet_csv_dir, exist_ok=True)
os.makedirs(mgc_dir, exist_ok=True)
os.makedirs(geo_parquet_unificado, exist_ok=True)

In [ ]:
# --------------------------------------------------
# 3. Descarga, Extracción y Conversión de Datos
# --------------------------------------------------

csv_files = []       # Rutas de CSV extraídos
parquet_geo_files = []   # Rutas de GeoParquet

# URLs base
base_url_csv = "https://www.inegi.org.mx/contenidos/programas/ccpv/2020/datosabiertos/ageb_manzana/"
base_url_shp = "https://www.inegi.org.mx/contenidos/productos/prod_serv/contenidos/espanol/bvinegi/productos/geografia/marcogeo/889463807469/"
shape_type = "m"  # Usado para manzanas

for num, geo_zip in zip(estados_num, estados_geo):
    estado_str = f"{num:02d}"
    print(f"\nProcesando estado {estado_str} ...")

    # --- Procesar CSV ---
    zip_csv_name = f"ageb_mza_urbana_{estado_str}_cpv2020_csv.zip"
    url_csv = base_url_csv + zip_csv_name
    zip_csv_path = download(url_csv, download_dir_csv)

    # Ruta interna y externa del CSV
    target_folder = f"ageb_mza_urbana_{estado_str}_cpv2020"
    csv_rel_path = os.path.join(target_folder, "conjunto_de_datos", f"conjunto_de_datos_ageb_urbana_{estado_str}_cpv2020.csv")
    csv_full_folder = os.path.join(csv_dir, target_folder)
    os.makedirs(csv_full_folder, exist_ok=True)
    csv_file_path = os.path.join(csv_dir, csv_rel_path)
    if not os.path.exists(csv_file_path):
        with ZipFile(zip_csv_path, 'r') as z:
            z.extract(csv_rel_path, csv_dir)
        print(f"CSV extraído para estado {estado_str}.")
    else:
        print(f"CSV para estado {estado_str} ya existe.")
    csv_files.append(csv_file_path)

    # --- Procesar Shapefile ---
    zip_shp_path = download(base_url_shp + geo_zip, mgc_dir)
    shp_state_dir = os.path.join(mgc_dir, "shp", estado_str)
    extract_shapefile([geo_zip], mgc_dir, shp_state_dir, shape_type)
    parquet_geo = convert_to_geoparquet(shp_state_dir, estado_str, shape_type)
    parquet_geo_files.append(parquet_geo)


In [ ]:
# --------------------------------------------------
# 4. Conversión de CSV a Parquet (manteniendo estructura de carpetas)
# --------------------------------------------------

for root, _, files in os.walk(csv_dir):
    for file in files:
        if file.lower().endswith(".csv"):
            csv_path = os.path.join(root, file)
            relative_path = os.path.relpath(csv_path, csv_dir)
            parquet_path = os.path.splitext(os.path.join(parquet_csv_dir, relative_path))[0] + ".parquet"
            os.makedirs(os.path.dirname(parquet_path), exist_ok=True)
            print(f"Convirtiendo {csv_path} a {parquet_path} ...")
            try:
                df = pd.read_csv(csv_path, low_memory=False, encoding='utf-8')
            except Exception as e:
                print(f"Error leyendo {csv_path} con utf-8: {e}. Se intentará con ISO-8859-1.")
                df = pd.read_csv(csv_path, low_memory=False, encoding='ISO-8859-1')
            df.to_parquet(parquet_path, index=False)
            print("¡Conversión completada!")


In [ ]:
parquet_path

In [ ]:
# --------------------------------------------------
# 5. Unificar GeoParquet en un único directorio
# --------------------------------------------------

for root, _, files in os.walk(os.path.join(mgc_dir, "shp")):
    for file in files:
        if file.lower().endswith(".geoparquet"):
            src_path = os.path.join(root, file)
            dst_path = os.path.join(geo_parquet_unificado, file)
            count = 1
            while os.path.exists(dst_path):
                name, ext = os.path.splitext(file)
                dst_path = os.path.join(geo_parquet_unificado, f"{name}_{count}{ext}")
                count += 1
            print(f"Copiando {src_path} a {dst_path} ...")
            shutil.copy2(src_path, dst_path)
print("Unificación de GeoParquet completada.")

In [ ]:
# --------------------------------------------------
# 5.1 Unificar Parquet CSV en un único directorio
# --------------------------------------------------

import os
import shutil

# Directorio fuente donde se generaron los Parquet a partir de los CSV
csv_parquet_source = "/content/inegi/ccpvagebmza/parquet"

# Directorio destino unificado
csv_parquet_unificado = "/content/inegi/ccpvagebmza/parquet_unificados"
os.makedirs(csv_parquet_unificado, exist_ok=True)

# Recorrer recursivamente el directorio fuente y copiar los archivos .parquet
for root, _, files in os.walk(csv_parquet_source):
    for file in files:
        if file.lower().endswith(".parquet"):
            src_path = os.path.join(root, file)
            dst_path = os.path.join(csv_parquet_unificado, file)
            # Evitar colisiones de nombres añadiendo un sufijo si es necesario
            count = 1
            while os.path.exists(dst_path):
                name, ext = os.path.splitext(file)
                dst_path = os.path.join(csv_parquet_unificado, f"{name}_{count}{ext}")
                count += 1
            print(f"Copiando {src_path} a {dst_path} ...")
            shutil.copy2(src_path, dst_path)

print("Copia de todos los archivos Parquet CSV en un solo nivel completada.")


In [ ]:
# --------------------------------------------------
# 6. Creación de Tablas en DuckDB
# --------------------------------------------------

# Definir archivo de base de datos
db_file = "datos_inegi_total_1.duckdb"
if os.path.exists(db_file):
    os.remove(db_file)
con = duckdb.connect(db_file)
con.execute("INSTALL spatial;")
con.execute("LOAD spatial;")

# Listas de archivos Parquet (CSV y GeoParquet)
csv_parquet_files = [
    os.path.join(csv_parquet_unificado, f)
    for f in os.listdir(csv_parquet_unificado)
    if f.lower().endswith(".parquet")
]
geo_parquet_files = [
    os.path.join(geo_parquet_unificado, f)
    for f in os.listdir(geo_parquet_unificado)
    if f.lower().endswith(".geoparquet")
]

# --- Crear tabla censo_all (datos censales) ---
con.execute("DROP TABLE IF EXISTS censo_all;")
union_queries = [f"SELECT * FROM read_parquet('{f}')" for f in csv_parquet_files]
query_censo = " UNION ALL ".join(union_queries)
con.execute(f"CREATE TABLE censo_all AS {query_censo};")
print("Tabla 'censo_all' creada con datos censales de todos los estados.")

# --- Crear tabla manzanas (datos geográficos) ---
con.execute("DROP TABLE IF EXISTS manzanas;")
union_queries = [f"SELECT * FROM read_parquet('{f}')" for f in geo_parquet_files]
query_manzanas = " UNION ALL ".join(union_queries)
con.execute(f"CREATE TABLE manzanas AS {query_manzanas};")
print("Tabla 'manzanas' creada con datos geográficos de todos los estados.")

In [ ]:
# --------------------------------------------------
# 7. Unir Datos Censales y Geográficos (censo_geo)
# --------------------------------------------------

# Se crea un campo CVEGEO a partir de los campos censales y se une con la geometría
con.execute("DROP TABLE IF EXISTS censo_geo;")
cols = [row[0] for row in con.execute("DESCRIBE censo_all;").fetchall()]
exclude = ['ENTIDAD', 'MUN', 'LOC', 'AGEB', 'MZA']
select_cols = [col for col in cols if col not in exclude]
select_cols_str = ', '.join(['c.' + col for col in select_cols])

con.execute(f"""
    CREATE TABLE censo_geo AS
    WITH censo AS (
        SELECT
            *,
            CONCAT(
                LPAD(CAST(ENTIDAD AS VARCHAR), 2, '0'),
                LPAD(CAST(MUN AS VARCHAR), 3, '0'),
                LPAD(CAST(LOC AS VARCHAR), 4, '0'),
                AGEB,
                LPAD(CAST(MZA AS VARCHAR), 3, '0')
            ) AS CVEGEO
        FROM censo_all
    ),
    shp AS (
        SELECT CVEGEO, geometry FROM manzanas
    )
    SELECT c.CVEGEO, {select_cols_str}, s.geometry
    FROM censo c
    LEFT JOIN shp s USING (CVEGEO)
    WHERE s.CVEGEO IS NOT NULL;
""")
print("Tabla 'censo_geo' creada uniendo datos censales y geográficos.")

In [ ]:
# --------------------------------------------------
# 8. Conversión de Campos a INTEGER en censo_geo (tabla censo_geo_int)
# --------------------------------------------------

fields_to_int = [
    "POBFEM", "POBMAS", "P_0A2", "P_0A2_F", "P_0A2_M", "P_3YMAS", "P_3YMAS_F", "P_3YMAS_M",
    "P_5YMAS", "P_5YMAS_F", "P_5YMAS_M", "P_12YMAS", "P_12YMAS_F", "P_12YMAS_M", "P_15YMAS", "P_15YMAS_F",
    "P_15YMAS_M", "P_18YMAS", "P_18YMAS_F", "P_18YMAS_M", "P_3A5", "P_3A5_F", "P_3A5_M", "P_6A11",
    "P_6A11_F", "P_6A11_M", "P_8A14", "P_8A14_F", "P_8A14_M", "P_12A14", "P_12A14_F", "P_12A14_M",
    "P_15A17", "P_15A17_F", "P_15A17_M", "P_18A24", "P_18A24_F", "P_18A24_M", "P_15A49_F", "P_60YMAS",
    "P_60YMAS_F", "P_60YMAS_M", "REL_H_M", "POB0_14", "POB15_64", "POB65_MAS", "PROM_HNV", "PNACENT",
    "PNACENT_F", "PNACENT_M", "PNACOE", "PNACOE_F", "PNACOE_M", "PRES2015", "PRES2015_F", "PRES2015_M",
    "PRESOE15", "PRESOE15_F", "PRESOE15_M", "P3YM_HLI", "P3YM_HLI_F", "P3YM_HLI_M", "P3HLINHE", "P3HLINHE_F",
    "P3HLINHE_M", "P3HLI_HE", "P3HLI_HE_F", "P3HLI_HE_M", "P5_HLI", "P5_HLI_NHE", "P5_HLI_HE", "PHOG_IND",
    "POB_AFRO", "POB_AFRO_F", "POB_AFRO_M", "PCON_DISC", "PCDISC_MOT", "PCDISC_VIS", "PCDISC_LENG",
    "PCDISC_AUD", "PCDISC_MOT2", "PCDISC_MEN", "PCON_LIMI", "PCLIM_CSB", "PCLIM_VIS", "PCLIM_HACO",
    "PCLIM_OAUD", "PCLIM_MOT2", "PCLIM_RE_CO", "PCLIM_PMEN", "PSIND_LIM", "P3A5_NOA", "P3A5_NOA_F",
    "P3A5_NOA_M", "P6A11_NOA", "P6A11_NOAF", "P6A11_NOAM", "P12A14NOA", "P12A14NOAF", "P12A14NOAM",
    "P15A17A", "P15A17A_F", "P15A17A_M", "P18A24A", "P18A24A_F", "P18A24A_M", "P8A14AN", "P8A14AN_F",
    "P8A14AN_M", "P15YM_AN", "P15YM_AN_F", "P15YM_AN_M", "P15YM_SE", "P15YM_SE_F", "P15YM_SE_M",
    "P15PRI_IN", "P15PRI_INF", "P15PRI_INM", "P15PRI_CO", "P15PRI_COF", "P15PRI_COM", "P15SEC_IN",
    "P15SEC_INF", "P15SEC_INM", "P15SEC_CO", "P15SEC_COF", "P15SEC_COM", "P18YM_PB", "P18YM_PB_F",
    "P18YM_PB_M", "GRAPROES", "GRAPROES_F", "GRAPROES_M", "PEA", "PEA_F", "PEA_M", "PE_INAC",
    "PE_INAC_F", "PE_INAC_M", "POCUPADA", "POCUPADA_F", "POCUPADA_M", "PDESOCUP", "PDESOCUP_F",
    "PDESOCUP_M", "PSINDER", "PDER_SS", "PDER_IMSS", "PDER_ISTE", "PDER_ISTEE", "PAFIL_PDOM",
    "PDER_SEGP", "PDER_IMSSB", "PAFIL_IPRIV", "PAFIL_OTRAI", "P12YM_SOLT", "P12YM_CASA", "P12YM_SEPA",
    "PCATOLICA", "PRO_CRIEVA", "POTRAS_REL", "PSIN_RELIG", "TOTHOG", "HOGJEF_F", "HOGJEF_M",
    "POBHOG", "PHOGJEF_F", "PHOGJEF_M", "TVIVHAB", "TVIVPAR", "VIVPAR_HAB", "VIVPARH_CV",
    "TVIVPARHAB", "VIVPAR_DES", "VIVPAR_UT", "OCUPVIVPAR", "PROM_OCUP", "PRO_OCUP_C", "VPH_PISODT",
    "VPH_PISOTI", "VPH_1DOR", "VPH_2YMASD", "VPH_1CUART", "VPH_2CUART", "VPH_3YMASC", "VPH_C_ELEC",
    "VPH_S_ELEC", "VPH_AGUADV", "VPH_AEASP", "VPH_AGUAFV", "VPH_TINACO", "VPH_CISTER", "VPH_EXCSA",
    "VPH_LETR", "VPH_DRENAJ", "VPH_NODREN", "VPH_C_SERV", "VPH_NDEAED", "VPH_DSADMA", "VPH_NDACMM",
    "VPH_SNBIEN", "VPH_REFRI", "VPH_LAVAD", "VPH_HMICRO", "VPH_AUTOM", "VPH_MOTO", "VPH_BICI",
    "VPH_RADIO", "VPH_TV", "VPH_PC", "VPH_TELEF", "VPH_CEL", "VPH_INTER", "VPH_STVP", "VPH_SPMVPI",
    "VPH_CVJ", "VPH_SINRTV", "VPH_SINLTC", "VPH_SINCINT", "VPH_SINTIC"
]
null_values = ['N/A', 'N/D', '*', '']

cols = [row[0] for row in con.execute("DESCRIBE censo_geo").fetchall()]
select_expr = []
for col in cols:
    if col in fields_to_int:
        expr = f"""CASE
            WHEN {col} IN ('N/A', 'N/D', '*', '') THEN NULL
            ELSE CAST({col} AS INTEGER)
        END AS {col}"""
        select_expr.append(expr)
    else:
        select_expr.append(col)
select_expr_str = ",\n".join(select_expr)

con.execute("DROP TABLE IF EXISTS censo_geo_int;")
con.execute(f"""
CREATE TABLE censo_geo_int AS
SELECT
    {select_expr_str}
FROM censo_geo;
""")
print("Tabla 'censo_geo_int' creada con campos numéricos convertidos a INTEGER donde corresponde.")

# (Opcional) Copiar censo_geo_int a una nueva base de datos
nuevo_db_file = "datos_censo_nacional.duckdb"
if os.path.exists(nuevo_db_file):
    os.remove(nuevo_db_file)
con.execute(f"ATTACH '{nuevo_db_file}' AS nuevo_db;")
con.execute("CREATE TABLE nuevo_db.censo_geo_int AS SELECT * FROM censo_geo_int;")
con.execute("DETACH nuevo_db;")
print(f"La tabla 'censo_geo_int' ha sido copiada a: {nuevo_db_file}")

In [ ]:
# --------------------------------------------------
# 9. Generación de la Grilla Hexagonal
# --------------------------------------------------

# Desconectar la conexión actual (si está abierta)
con.close()

# Conectar a la base de datos 'datos_censo_nacional.duckdb'
nuevo_db_file = "datos_censo_nacional.duckdb"
con_nuevo = duckdb.connect(nuevo_db_file)
con_nuevo.execute("INSTALL spatial;")
con_nuevo.execute("LOAD spatial;")


# Extraer centroides de las geometrías de censo_geo_int en formato WKB
query = """
SELECT
    ST_AsWKB(ST_Centroid(geometry)) AS geometry
FROM censo_geo_int;
"""
df = con_nuevo.execute(query).fetchdf()
df["geometry"] = df["geometry"].apply(lambda x: wkb.loads(bytes(x)) if x is not None else None)
gdf_centroids = gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")
print("Centroides extraídos:", gdf_centroids.head())

# Reproyectar a EPSG:6372 y generar grilla hexagonal con circumradio 1000
gdf_centroids_6372 = gdf_centroids.to_crs(epsg=6372)
hex_grid = ghg.make_grid_from_gdf(gdf_centroids_6372, R=1000)
hex_grid_4326 = hex_grid.to_crs(epsg=4326)
print("Grid hexagonal generado:", hex_grid_4326.head())

# Exportar grid a Shapefile y Parquet
hex_grid_4326.to_file("grid_nacional.shp")
grid_parquet_file = "hex_grid_1km_4326.parquet"
hex_grid_4326.to_parquet(grid_parquet_file)


# Asumimos que 'grid_parquet_file' contiene el nombre del archivo Parquet con el grid hexagonal
grid_parquet_file = "hex_grid_1km_4326.parquet"

# Crear (o reemplazar) la tabla 'hex_grid_1km_4326' en la nueva base de datos
con_nuevo.execute(f"""
CREATE OR REPLACE TABLE hex_grid_1km_4326 AS
SELECT * FROM read_parquet('{grid_parquet_file}')
""")

# Mostrar información de la tabla y algunos registros para verificar
df_info = con_nuevo.execute("PRAGMA table_info('hex_grid_1km_4326');").fetchdf()
print("Información de la tabla hex_grid_1km_4326:")
print(df_info)

df_first10 = con_nuevo.execute("SELECT * FROM hex_grid_1km_4326 LIMIT 10;").fetchdf()
print("Primeros 10 registros de hex_grid_1km_4326:")
print(df_first10)

con_nuevo.close()

In [ ]:
import duckdb
import geopandas as gpd
import shapely
import matplotlib.pyplot as plt
import numpy as np

# Conectar a la base de datos
with duckdb.connect("/content/datos_censo_nacional.duckdb") as con:  # Usar 'with' para asegurar cierre
    con.execute("INSTALL spatial;")
    con.execute("LOAD spatial;")

    con.execute("DROP TABLE IF EXISTS censo_centroids;")

    # Precalcular centroides y crear tabla/índices
    con.execute("""
        CREATE TABLE censo_centroids AS
        SELECT
            *,  -- Selecciona las columnas necesarias
            ST_Centroid(geometry) AS centroid
        FROM censo_geo_int;
    """)

    con.execute("DROP INDEX IF EXISTS idx_censo_centroid;")
    con.execute("DROP INDEX IF EXISTS idx_hex_geom;")
    con.execute("CREATE INDEX idx_censo_centroid ON censo_centroids USING RTREE(centroid);")
    con.execute("CREATE INDEX idx_hex_geom ON hex_grid_1km_4326 USING RTREE(geometry);")

    # Crear la tabla hex_final directamente en DuckDB (CTAS)
    con.execute("DROP TABLE IF EXISTS hex_final;")
    con.execute("""
    CREATE TABLE hex_final AS
    SELECT
        h.cell_id,
        ST_AsWKB(h.geometry) AS geometry,
        SUM(c.POBTOT) AS PoblacionTotal,
        SUM(c.VPH_INTER) AS ViviendasInternet,
        SUM(c.VPH_CVJ) AS ViviendasConsola,
        SUM(c.VPH_SPMVPI) AS ViviendasStreaming
    FROM hex_grid_1km_4326 h
    LEFT JOIN censo_centroids c
        ON ST_Intersects(c.centroid, h.geometry)  -- Usar centroide precalculado
    GROUP BY h.cell_id, h.geometry;
    """)

    # Cargar a GeoDataFrame (si es necesario)
    hex_final_df = con.execute("SELECT * FROM hex_final").fetchdf()
    hex_final_df["geometry"] = hex_final_df["geometry"].apply(lambda x: wkb.loads(bytes(x)) if x is not None else None)
    hex_final = gpd.GeoDataFrame(hex_final_df, geometry="geometry", crs="EPSG:4326")


    # Graficar (sin cambios)
    variable = 'PoblacionTotal'
    cmap = 'turbo'
    fig, ax = plt.subplots(1, 1, figsize=(20, 20))
    hex_final.plot(column=variable, cmap=cmap, linewidth=0.8, ax=ax, edgecolor='0.8')
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=hex_final[variable].min(), vmax=hex_final[variable].max()))
    sm._A = []
    cbar = fig.colorbar(sm, ax=ax)
    plt.show()

# La conexión se cierra automáticamente gracias al 'with'

In [ ]:
import geopandas as gpd
import folium  # Asegúrate de importar folium
import json

variable = 'PoblacionTotal'  # Usar el nombre correcto de la columna

# Añade una columna 'index' para usarla en el key_on de Folium
# Usa .copy() para evitar SettingWithCopyWarning
hex_final = hex_final.copy()
hex_final['index'] = hex_final.index.astype(str)


# Convierte el GeoDataFrame a JSON
gdf_json = hex_final.to_json()

# Calcula el centro del mapa usando el centroide de los polígonos
centroid = hex_final.geometry.centroid
avg_lat = centroid.y.mean()
avg_lon = centroid.x.mean()

# Crea el objeto de mapa de Folium
m = folium.Map(location=[avg_lat, avg_lon], zoom_start=13)

# Agrega la capa Choropleth
folium.Choropleth(
    geo_data=gdf_json,
    name='choropleth',
    data=hex_final,
    columns=['index', variable],  # Usar 'index' y el nombre correcto de la variable
    key_on='feature.id',  # Usar feature.id, ya que to_json() lo asigna automáticamente
    fill_color='YlGnBu',
    fill_opacity=0.55,
    line_opacity=0.1,
    legend_name=variable
).add_to(m)

# Agrega control de capas (opcional)
folium.LayerControl().add_to(m)

# Muestra el mapa
m